In [1]:
# Dependencies
import pandas as pd
import requests
from selenium import webdriver
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
# url for Nasa
url = 'http://mars.nasa.gov/news'


In [3]:
# splinter setup
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [4]:
# Beautiful soupify it
html = browser.html
soup = bs(html, 'lxml')


In [5]:
# validate soup object
# print(soup.prettify())

In [6]:
# extract the title and the teaser body
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

NASA's MAVEN Spacecraft Finds That "Stolen" Electrons Enable Unusual Aurora on Mars
Auroras appear on Earth as ghostly displays of colorful light in the night sky, usually near the poles.


In [7]:
# jpl - feature image
# setup url and base url
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
jpl_base_url = 'https://www.jpl.nasa.gov'

In [8]:
# splinter to get image
browser.visit(jpl_url)

In [9]:
# splinterizing and soupifying
jpl_html = browser.html
jpl_soup = bs(jpl_html, 'lxml')

In [10]:
# validate
# print(jpl_soup.prettify())

In [11]:
# get the featured image
featured_img = jpl_base_url + jpl_soup.find(id='full_image')['data-fancybox-href']

In [12]:
# validate
featured_img

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA09320_ip.jpg'

In [13]:
# twitter url to visit
twitter_url = 'https://twitter.com/marswxreport?lang=en'

# splinter to get object
browser.visit(twitter_url)

# splinterizing and soupifying
twitter_html = browser.html
twitter_soup = bs(twitter_html, 'lxml')



In [14]:
# validate
# print(twitter_soup.prettify())

In [15]:
# get the tweet text
mars_weather = twitter_soup.find(class_='tweet-text').text

In [16]:
# validate
print(mars_weather)

L-2 years. #Mars2020


In [17]:
# url to visit
facts_url = 'http://space-facts.com/mars/'

# scrape table from html
table = pd.read_html(facts_url)

# create df with columns' headers
temp_df = table[0]
temp_df.columns = ['Label','Value']


In [18]:
df = temp_df.set_index('Label')
df

,Value
Label,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [19]:
# convert to html table
facts_table = df.to_html()


In [20]:
# validate
# facts_table

In [21]:
# USGS Astrogeology url
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# splinter to get object
browser.visit(usgs_url)

# splinterizing and soupifying
usgs_html = browser.html
usgs_soup = bs(usgs_html, 'lxml')

In [22]:
# print(usgs_soup.prettify())

In [23]:
# use css notation with select() to get all items
items = usgs_soup.select('div.item')

In [27]:
# Validate
# print(items)

In [28]:
# setup image url list
image_urls = []

# loop through every item of html, parse out title and get url for image
for item in items:
    
    # extract the title and truncate
    title = (item.find('h3').text).replace(' Enhanced', '')
        
    # splinterize: use splinter to click on the link
    # (http://splinter.readthedocs.io/en/latest/elements-in-the-page.html)
    browser.click_link_by_partial_text(title)
    
    # soupify
    soup = bs(browser.html, 'html.parser')
    
    # find the image anchor by text attribute
    img = soup.find('a', text='Sample')
    
    # extract url from img
    img_url = img['href']
    
    #make a dict and append to the list
    image_urls.append({'title': title, 'image_url': img_url})


In [29]:
image_urls

[{'title': 'Cerberus Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_unenhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_unenhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'image_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_unenhanced.tif/full.jpg'}]